<img src="https://5264302.fs1.hubspotusercontent-na1.net/hubfs/5264302/Demo%20Asset%20Resources/CM-Demo-market_data_overview-Cover.png" width=1100 margin-left='auto' margin-right='auto'/>

Coin Metrics **Market Data Feed** provides access to historical and real-time data from over 39 of the world’s leading spot and derivatives crypto exchanges. We offer all of the fundamental market-related data types including tick-by-tick trades, quotes, order book snapshots, candles, and more.

The example charts showcased in this notebook are presented on a weekly basis in our [State of the Market](http://stateofthe.market/) newsletter.

### Resources

This notebook demonstrates basic functionality offered by the Coin Metrics Python API Client and [Market Data Feed](https://coinmetrics.io/market-data-feed/).

Coin Metrics offers a vast assortment of data for hundreds of cryptoassets. The Python API Client allows for easy access to this data using Python without needing to create your own wrappers using `requests` and other such libraries.

To understand the data that Coin Metrics offers, feel free to peruse the resources below.

- The [Coin Metrics API v4](https://docs.coinmetrics.io/api/v4) website contains the full set of endpoints and data offered by Coin Metrics.
- The [Coin Metrics Product Documentation](https://docs.coinmetrics.io/info) gives detailed, conceptual explanations of the data that Coin Metrics offers.
- The [API Spec](https://coinmetrics.github.io/api-client-python/site/api_client.html) contains a full list of functions.

## Setup

In [1]:
from os import environ
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import logging
from datetime import date, datetime, timedelta
from coinmetrics.api_client import CoinMetricsClient
import json
import logging
from pytz import timezone as timezone_conv
from datetime import timezone as timezone_info
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
sns.set_theme()
sns.set(rc={'figure.figsize':(12,8)})

In [3]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [4]:
# We recommend privately storing your API key in your local environment.
try:
    api_key = environ["CM_API_KEY"]
    logging.info("Using API key found in environment")
except KeyError:
    api_key = ""
    logging.info("API key not found. Using community client")
client = CoinMetricsClient(api_key)

2022-09-08 09:24:16 INFO     Using API key found in environment


# Catalog Endpoints

The `catalog` endpoints display the set of data available to your API key. The `catalog-all` endpoints display the full set of data for CM Pro users.

In [5]:
btc_market_catalog = client.catalog_markets(
                    base='btc',
                    market_type='spot',
                    exchange='binance'
                    ).to_dataframe()
btc_market_catalog.tail(5)

,market,min_time,max_time,exchange,type,orderbooks,quotes,base,quote,min_time_trades,max_time_trades
17,binance-btc-usds_stableusd-spot,2019-02-06 03:01:56.311000+00:00,2020-02-20 07:53:26.045000+00:00,binance,spot,NaN,NaN,btc,usds_stableusd,2019-02-06T03:01:56.311000000Z,2020-02-20T07:53:26.045000000Z
18,binance-btc-usdt-spot,2017-08-17 04:00:32.285000+00:00,2022-09-08 13:15:28.208000+00:00,binance,spot,"{'min_time': '2021-08-03T16:00:00.000000000Z',...","{'min_time': '2021-08-03T16:00:00.000000000Z',...",btc,usdt,2017-08-17T04:00:32.285000000Z,2022-09-08T13:15:28.208000000Z
19,binance-btc-ust-spot,2022-04-22 09:00:00+00:00,2022-05-13 00:49:59.802000+00:00,binance,spot,"{'min_time': '2022-04-22T09:00:00.000000000Z',...","{'min_time': '2022-04-22T09:00:00.000000000Z',...",btc,ust,2022-04-22T09:00:00.802000000Z,2022-05-13T00:49:59.802000000Z
20,binance-btc-vai-spot,2021-02-04 10:00:00.761000+00:00,2021-09-18 12:00:00+00:00,binance,spot,"{'min_time': '2021-09-14T16:00:00.000000000Z',...","{'min_time': '2021-09-14T16:00:00.000000000Z',...",btc,vai,2021-02-04T10:00:00.761000000Z,2021-09-18T11:55:27.171000000Z
21,binance-btc-zar-spot,2020-03-27 04:01:58.397000+00:00,2021-04-02 09:59:29.746000+00:00,binance,spot,NaN,NaN,btc,zar,2020-03-27T04:01:58.397000000Z,2021-04-02T09:59:29.746000000Z


Catalog objects return a list of dictionaries. For `catalog_assets`, each element of the list is an asset, while each dictionary is a set of metadata for that specific asset.

In [6]:
print(f"Market catalog metadata includes: {list(btc_market_catalog.keys())}")

Market catalog metadata includes: ['market', 'min_time', 'max_time', 'exchange', 'type', 'orderbooks', 'quotes', 'base', 'quote', 'min_time_trades', 'max_time_trades']


---
# Example 1: Returns by coin in the CM Reference Rates universe
---
We offer reference rates quoted in USD, Euro, Bitcoin, and Ethereum. We now support these quote currencies for our entire reference rates coverage universe of over 500 assets and for all of our frequencies, including 1s, 1m, 1h, 1d-ny-close and 1d.
    
**Fig. 1 - 7 Day Price Change chart from State of the Market**    
<img src="https://5264302.fs1.hubspotusercontent-na1.net/hubfs/5264302/State%20of%20the%20Market%20-%20Chart%20Examples/Mini-RefRate-Returns.png" width=450 margin-left='auto' margin-right='auto'/>


In [7]:
# Get all assets that have a reference rate 
assets_refrate = client.catalog_metrics("ReferenceRateUSD").to_dataframe()
assets_refrate[['metric','frequency','asset']]

,metric,frequency,asset
0,ReferenceRateUSD,1d,1inch
1,ReferenceRateUSD,1d,aave
2,ReferenceRateUSD,1d,abbc
3,ReferenceRateUSD,1d,abt
4,ReferenceRateUSD,1d,aca
...,...,...,...
2615,ReferenceRateUSD,1s,zec
2616,ReferenceRateUSD,1s,zen
2617,ReferenceRateUSD,1s,zil
2618,ReferenceRateUSD,1s,zks


In [8]:
print('\nNumber of unique Reference Rate assets: ' + str(len(pd.unique(assets_refrate['asset'])))+'\n')


Number of unique Reference Rate assets: 527



We can retrieve Reference Rates from the `get_asset_metrics` endpoint. The code snippets below demonstrate how to do this with a small list of assets. 

In [9]:
# Retrieve Reference Rate
df_prices = client.get_asset_metrics(
    assets=['btc', 'eth', 'bnb', 'ada', 'doge', 'xrp'],
    metrics='ReferenceRateUSD',
    frequency='1d',
    start_time='2022-08-10',
    end_time='2022-08-17'
).to_dataframe()

# Assign datatypes
df_prices["time"] = pd.to_datetime(df_prices.time)
df_prices["ReferenceRateUSD"] = df_prices.ReferenceRateUSD.astype(float)
# Reshape dataset so assets are in columns, dates are the rows, and the values are prices
df_prices_pivot = df_prices.pivot(index="time",columns="asset",values="ReferenceRateUSD")

In [10]:
df_prices_pivot.head(3)

asset,ada,bnb,btc,doge,eth,xrp
time,,,,,,
2022-08-10 00:00:00+00:00,0.514033,325.442291,23186.291746,0.069148,1703.992249,0.368363
2022-08-11 00:00:00+00:00,0.536694,328.034925,23923.058483,0.071153,1850.829961,0.380704
2022-08-12 00:00:00+00:00,0.530497,323.337253,23934.439056,0.070800,1878.113096,0.379859


In [11]:
# Index each asset's time series to 1 
for col in df_prices_pivot.columns:
    logging.info(f"Calculating returns for {col}....")
    first_price = df_prices_pivot[df_prices_pivot[col].notnull()][col].iloc[0]
    df_prices_pivot[col] = df_prices_pivot[col]/first_price
    df_prices_pivot[col] = df_prices_pivot[col].ffill()

2022-09-08 09:24:17 INFO     Calculating returns for ada....
2022-09-08 09:24:17 INFO     Calculating returns for bnb....
2022-09-08 09:24:17 INFO     Calculating returns for btc....
2022-09-08 09:24:17 INFO     Calculating returns for doge....
2022-09-08 09:24:17 INFO     Calculating returns for eth....
2022-09-08 09:24:17 INFO     Calculating returns for xrp....


In [12]:
df_prices_pivot

asset,ada,bnb,btc,doge,eth,xrp
time,,,,,,
2022-08-10 00:00:00+00:00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2022-08-11 00:00:00+00:00,1.044084,1.007966,1.031776,1.028991,1.086173,1.033501
2022-08-12 00:00:00+00:00,1.032028,0.993532,1.032267,1.023896,1.102184,1.031207
2022-08-13 00:00:00+00:00,1.051484,1.005325,1.052187,1.046478,1.147176,1.030523
2022-08-14 00:00:00+00:00,1.089703,0.995682,1.053477,1.054227,1.162703,1.025361
2022-08-15 00:00:00+00:00,1.110819,0.976807,1.048646,1.183001,1.136355,1.021364
2022-08-16 00:00:00+00:00,1.070279,0.980408,1.038715,1.106155,1.116053,1.017113
2022-08-17 00:00:00+00:00,1.085200,0.971739,1.029437,1.257445,1.101305,1.023202


---
# Example 2: Spot trading volume on Coinbase
---
**Fig. 2 - Coinbase daily spot volume breakdown from State of the Market**    
<img src="https://5264302.fs1.hubspotusercontent-na1.net/hubfs/5264302/State%20of%20the%20Market%20-%20Chart%20Examples/Coinbase-Volume-Breakdown.png" width=1100 margin-left='auto' margin-right='auto'/>

### Foundational Data Types - Trades

Trades are one of the foundational data types we collect from exchanges. From raw trades data, we can construct additional aggregated metrics.

In [13]:
coinbase_btc_trades = client.get_market_trades(
    markets='coinbase-btc-usd-spot',
    limit_per_market=5,
    paging_from='end'
).to_dataframe()

In [14]:
coinbase_btc_trades

,market,time,coin_metrics_id,amount,price,database_time,side
0,coinbase-btc-usd-spot,2022-09-08 13:24:16.160421+00:00,407674837,0.019287,19095.6,2022-09-08 13:24:16.275800+00:00,sell
1,coinbase-btc-usd-spot,2022-09-08 13:24:16.160421+00:00,407674838,0.0001,19095.6,2022-09-08 13:24:16.275800+00:00,sell
2,coinbase-btc-usd-spot,2022-09-08 13:24:16.160421+00:00,407674839,0.0001,19095.6,2022-09-08 13:24:16.275800+00:00,sell
3,coinbase-btc-usd-spot,2022-09-08 13:24:16.868614+00:00,407674840,0.000051,19095.84,2022-09-08 13:24:16.996622+00:00,buy
4,coinbase-btc-usd-spot,2022-09-08 13:24:17.707182+00:00,407674841,0.000153,19096.32,2022-09-08 13:24:17.857398+00:00,buy


### Spot Volume Share - Candles Data

From raw trades data, we construct OHLC candles for each market. For our *Spot Volume % by Asset* chart, we derive volume from our `get_market_candles` endpoint.

All of our endpoints that accept the markets parameter will accept wildcards  like exchange-* or exchange-*-spot or *USDT-future. The wildcards will match any market which fits this pattern so users do not need to specify every individual market when querying data for multiple markets. 

In [15]:
candles_coinbase = client.get_market_candles(
    markets="coinbase-*-spot", # wildcards can be passed to get all asset pairs
    start_time="2022-08-16",
    end_time="2022-08-17",
    frequency="1d"
).to_dataframe()
candles_coinbase["candle_usd_volume"] = candles_coinbase.candle_usd_volume.astype(float)
candles_coinbase["time"] = pd.to_datetime(candles_coinbase.time)

In [16]:
candles_coinbase.head()

,market,time,price_open,price_close,price_high,price_low,vwap,volume,candle_usd_volume,candle_trades_count
0,coinbase-1inch-btc-spot,2022-08-16 00:00:00+00:00,0.000034,0.000034,0.000035,0.000034,0.000034,12601.82,10324.267565,204
1,coinbase-1inch-btc-spot,2022-08-17 00:00:00+00:00,0.000034,0.000033,0.000035,0.000033,0.000034,6400.08,5251.890725,226
2,coinbase-1inch-eur-spot,2022-08-16 00:00:00+00:00,0.807,0.805,0.82,0.795,0.807773,62791.36,51503.118776,475
3,coinbase-1inch-eur-spot,2022-08-17 00:00:00+00:00,0.805,0.755,0.828,0.752,0.78722,56349.99,45122.297032,478
4,coinbase-1inch-gbp-spot,2022-08-16 00:00:00+00:00,0.679,0.677,0.691,0.669,0.680058,16631.32,13651.390357,67


* **price_open:**   The opening price of the candle.
* **price_high:**  The high price of the candle.
* **price_low:**  The low price of the candle.
* **price_close:** The close price of the candle.
* **vwap:**  The volume-weighted average price of the candle.
* **volume:** The volume of the candle in units of the base asset.
* **candle_usd_volume:** The volume of the candle in units of U.S. dollars. 
* **candle_trades_count:** The number of trades in the candle interval. 

### Total Exchange Spot Volume - Exchange Metrics

We can retrieve the overall volume on the exchange using our `exchange_metrics` endpoint.

In [17]:
coinbase_volume = client.get_exchange_metrics(
    exchanges='coinbase',
    metrics ='volume_reported_spot_usd_1d',
    start_time='2022-08-10',
    end_time='2022-08-17'
).to_dataframe()

In [18]:
coinbase_volume

,exchange,time,volume_reported_spot_usd_1d
0,coinbase,2022-08-10 00:00:00+00:00,2360484419.87473
1,coinbase,2022-08-11 00:00:00+00:00,2382406446.43788
2,coinbase,2022-08-12 00:00:00+00:00,1565909356.21818
3,coinbase,2022-08-13 00:00:00+00:00,1561807128.91978
4,coinbase,2022-08-14 00:00:00+00:00,2089614317.84876
5,coinbase,2022-08-15 00:00:00+00:00,2510659692.74971
6,coinbase,2022-08-16 00:00:00+00:00,1975114962.03532
7,coinbase,2022-08-17 00:00:00+00:00,2347289496.22169


---
# Example 3: Spot order book depth
---
**Fig. 3 - Binance order book depth from State of the Market**    
<img src="https://5264302.fs1.hubspotusercontent-na1.net/hubfs/5264302/State%20of%20the%20Market%20-%20Chart%20Examples/Binance-Order-Book-Depth.png" width=900 margin-left='auto' margin-right='auto'/>

### Foundational Data Types - Order Book Snapshots

Exchange order book data is one of the most foundational data types in the crypto industry— arguably, even more foundational than trades data, as two orders must be matched for a trade to occur. Order book data is useful for various entities, including  market makers, systematic or quantitative traders, and funds studying trade execution patterns.

Coin Metrics stores three types of order book snapshots. One type consists of a snapshot of the top 100 bids and top 100 asks taken once every 10 seconds for major markets. The second type consists of a full order book snapshot (every bid and every ask) taken once every hour for all markets. The third is a snapshot where the price is +/-10% of mid-price taken once every 10 seconds. All of these snapshots are served through our */timeseries/market-orderbooks* endpoint.

In [19]:
top100_snapshot = client.get_market_orderbooks(
    markets='coinbase-btc-usd-spot',
    depth_limit=100, # For full order book depth, pass the parameter 'full_book'. 
                     # For 10% of depth (where the price is within +/-10 % of mid-price), pass the parameter '10pct_mid_price'.
    limit_per_market=5,
    paging_from='end'
).to_dataframe()

In [20]:
top100_snapshot

,market,time,coin_metrics_id,asks,bids,database_time
0,coinbase-btc-usd-spot,2022-09-08 13:14:30+00:00,1662642870000000-0,"[{'price': '19076.29', 'size': '0.001'}, {'pri...","[{'price': '19073.8', 'size': '0.0010939'}, {'...",2022-09-08 13:23:31.558617+00:00
1,coinbase-btc-usd-spot,2022-09-08 13:14:40+00:00,1662642880000000-0,"[{'price': '19077.3', 'size': '0.25'}, {'price...","[{'price': '19074.63', 'size': '0.00220184'}, ...",2022-09-08 13:23:41.720565+00:00
2,coinbase-btc-usd-spot,2022-09-08 13:14:50+00:00,1662642890000000-0,"[{'price': '19076.51', 'size': '0.25'}, {'pric...","[{'price': '19072.35', 'size': '0.00196169'}, ...",2022-09-08 13:23:51.782668+00:00
3,coinbase-btc-usd-spot,2022-09-08 13:15:00+00:00,1662642900000000-0,"[{'price': '19059.55', 'size': '0.00630529'}, ...","[{'price': '19057.24', 'size': '0.001'}, {'pri...",2022-09-08 13:24:11.944709+00:00
4,coinbase-btc-usd-spot,2022-09-08 13:15:10+00:00,1662642910000000-0,"[{'price': '19053.77', 'size': '0.00658501'}, ...","[{'price': '19050.27', 'size': '0.25'}, {'pric...",2022-09-08 13:24:22.031015+00:00


In [21]:
bids = eval(top100_snapshot.bids[0])
bids[0:10]

[{'price': '19073.8', 'size': '0.0010939'},
 {'price': '19073.78', 'size': '0.05'},
 {'price': '19073.41', 'size': '0.004'},
 {'price': '19073.06', 'size': '0.0002'},
 {'price': '19073.05', 'size': '0.52429974'},
 {'price': '19072.23', 'size': '0.03471869'},
 {'price': '19072.22', 'size': '0.03349536'},
 {'price': '19072.06', 'size': '0.0045692'},
 {'price': '19072.05', 'size': '0.02920172'},
 {'price': '19072.03', 'size': '0.20920236'}]

### Market Quotes - Best Bid & Asks

As an added convenience, we also serve the top bid/ask via a separate timeseries/market-quotes endpoint. Quotes are derived from our order book snapshots, so they are available at the same 10s intervals.

In [22]:
btc_quotes = client.get_market_quotes(
    markets='coinbase-btc-usd-spot',
    limit_per_market=5,
    paging_from='end'
).to_dataframe()

In [23]:
btc_quotes

,market,time,coin_metrics_id,ask_price,ask_size,bid_price,bid_size
0,coinbase-btc-usd-spot,2022-09-08 13:14:30+00:00,1662642870000000-0,19076.29,0.001,19073.8,0.001094
1,coinbase-btc-usd-spot,2022-09-08 13:14:40+00:00,1662642880000000-0,19077.3,0.25,19074.63,0.002202
2,coinbase-btc-usd-spot,2022-09-08 13:14:50+00:00,1662642890000000-0,19076.51,0.25,19072.35,0.001962
3,coinbase-btc-usd-spot,2022-09-08 13:15:00+00:00,1662642900000000-0,19059.55,0.006305,19057.24,0.001
4,coinbase-btc-usd-spot,2022-09-08 13:15:10+00:00,1662642910000000-0,19053.77,0.006585,19050.27,0.25


**Note:** We now also offer *every quote update* via the new Coin Metrics flat file application.

---
# Example 4: Futures data types
---
We offer futures data for 3,000+ markets across top derivatives trading venues such as Binance, CME, FTX, BitMEX, Huobi, Bybit, etc. Supported data types include liquidations, contract prices, open interest, candles, volume, funding rates, and more.

**Fig. 4 - Bitcoin and Ethereum futures open interest from State of the Market**    
<img src="https://5264302.fs1.hubspotusercontent-na1.net/hubfs/5264302/State%20of%20the%20Market%20-%20Chart%20Examples/BTC-ETH-open-interest.png" width=550 margin-left='auto' margin-right='auto'/>



### Market Open Interest - Total Contracts Outstanding
Open interest represents the number of contracts that are currently outstanding and not settled for a specific derivatives market. 

In [24]:
oi_btc_perp = client.get_market_open_interest(
    markets='binance-BTCUSD_PERP-future',
    end_time='2022-08-17',
    limit_per_market=5,
    paging_from='end'
).to_dataframe()

In [25]:
oi_btc_perp

,market,time,contract_count,value_usd,database_time,exchange_time
0,binance-BTCUSD_PERP-future,2022-08-17 23:55:00+00:00,3982211,398221100,2022-08-17 23:55:22.917770+00:00,2022-08-17 23:55:00+00:00
1,binance-BTCUSD_PERP-future,2022-08-17 23:56:00+00:00,3981078,398107800,2022-08-17 23:56:21.928029+00:00,2022-08-17 23:56:00+00:00
2,binance-BTCUSD_PERP-future,2022-08-17 23:57:00+00:00,3979200,397920000,2022-08-17 23:57:35.499955+00:00,2022-08-17 23:57:00+00:00
3,binance-BTCUSD_PERP-future,2022-08-17 23:58:00+00:00,3979178,397917800,2022-08-17 23:58:46.517519+00:00,2022-08-17 23:58:00+00:00
4,binance-BTCUSD_PERP-future,2022-08-17 23:59:00+00:00,3978605,397860500,2022-08-17 23:59:10.522441+00:00,2022-08-17 23:59:00+00:00


### Aggregated Open Interest - Daily by Asset & Contract Type

In addition to querying open interest for specific markets/contracts, the `get_asset_metrics` endpoint can also be used to retrieve aggregated open interest. Our reported future open interest metric is an aggregation of the reported future open interest from all futures exchanges in CM's coverage universe.

We offer aggregated futures open interest for the following futures contract types:
- Reported Future Open Interest
- Reported Perpetual Future Open Interest
- Reported Non-Perpetual Future Open Interest
- Reported Coin-Margined Future Open Interest
- Reported Tether-Margined Future Open Interest

In [26]:
oi_btc_eth = client.get_asset_metrics(
    assets='btc,eth',
    metrics='open_interest_reported_future_usd,open_interest_reported_future_nonperpetual_usd,open_interest_reported_future_perpetual_usd',
    frequency='1d',
    limit_per_asset=3,
    paging_from='end'
).to_dataframe()

In [27]:
oi_btc_eth

,asset,time,open_interest_reported_future_nonperpetual_usd,open_interest_reported_future_perpetual_usd,open_interest_reported_future_usd
0,btc,2022-09-06 00:00:00+00:00,3862228970.93802,8782116033.80155,12644345004.739599
1,btc,2022-09-07 00:00:00+00:00,3837050416.74342,8374689456.63553,12211739873.378901
2,btc,2022-09-08 00:00:00+00:00,3821353967.66649,8335924305.19914,12157278272.865601
3,eth,2022-09-06 00:00:00+00:00,2592942895.66542,6754204726.94287,9347147622.60829
4,eth,2022-09-07 00:00:00+00:00,2524540158.17145,6149356724.58989,8673896882.761339
5,eth,2022-09-08 00:00:00+00:00,2582164736.76738,6544805495.76107,9126970232.52845


### Perpetual Futures Funding Rates
Funding rates are a mechanism that exchanges use to ensure that perpetual futures trade at a price that is close to the price of the underlying spot markets. The funding rate is used to calculate the funding fee which long position holders pay short position holders, or vice versa, as a way to incentivize market participants to take positions that keep perpetual futures prices close to the underlying. 

In [28]:
funding_btc_perp = client.get_market_funding_rates(
    markets='binance-BTCUSD_PERP-future',
    end_time='2022-08-17',
    limit_per_market=5
).to_dataframe()

In [29]:
funding_btc_perp

,market,time,database_time,rate,period,interval
0,binance-BTCUSD_PERP-future,2020-08-10 16:00:00+00:00,2020-12-02 10:49:37.530167+00:00,0.0001,08:00:00,08:00:00
1,binance-BTCUSD_PERP-future,2020-08-11 00:00:00+00:00,2020-12-02 10:49:37.530167+00:00,0.0001,08:00:00,08:00:00
2,binance-BTCUSD_PERP-future,2020-08-11 08:00:00.008000+00:00,2020-12-02 10:49:37.530167+00:00,0.0001,08:00:00,08:00:00
3,binance-BTCUSD_PERP-future,2020-08-11 16:00:00+00:00,2020-12-02 10:49:37.530167+00:00,0.0001,08:00:00,08:00:00
4,binance-BTCUSD_PERP-future,2020-08-12 00:00:00+00:00,2020-12-02 10:49:37.530167+00:00,0.0001,08:00:00,08:00:00


**Fig. 5 - Bitcoin perpetual futures funding rates from State of the Market**    
<img src="https://5264302.fs1.hubspotusercontent-na1.net/hubfs/5264302/State%20of%20the%20Market%20-%20Chart%20Examples/BTC-perp-funding-rates.png" width=850 margin-left='auto' margin-right='auto'/>

### Futures Basis Metrics - Aggregated by Exchange-Asset
The basis is the annualized percent difference between the price of a theoretical futures contract and the price of its underlying spot market. Coin Metrics calculates this for several exchange-assets such as `binance-btc` and `ftx-eth`. We calculate four basis metrics at defined days to expiration: 30 day, 60 day, 90 day, and 120 day.

In [30]:
basis_binance = client.get_exchange_asset_metrics(
    exchange_assets='binance-btc', 
    metrics='basis_annualized_30d_exp,basis_annualized_60d_exp,basis_annualized_90d_exp'
).to_dataframe()

In [31]:
basis_binance.tail()

,exchange_asset,time,basis_annualized_30d_exp,basis_annualized_60d_exp,basis_annualized_90d_exp
814,binance-btc,2022-09-04 00:00:00+00:00,-0.005188,0.001965,0.004349
815,binance-btc,2022-09-05 00:00:00+00:00,-0.001364,0.002653,0.003993
816,binance-btc,2022-09-06 00:00:00+00:00,0.007337,0.008232,0.00853
817,binance-btc,2022-09-07 00:00:00+00:00,-0.012086,-0.003326,-0.000406
818,binance-btc,2022-09-08 00:00:00+00:00,-0.000113,0.004746,0.006366


**Fig. 6 - Bitcoin futures basis from State of the Market**    
<img src="https://5264302.fs1.hubspotusercontent-na1.net/hubfs/5264302/State%20of%20the%20Market%20-%20Chart%20Examples/Futures-Basis.png" width=550 margin-left='auto' margin-right='auto'/>

### Futures Liquidations - Individual Orders/Trades
Exchanges which offer futures markets utilize a risk management system that will attempt to close a user’s position before the point at which the user begins to owe more than what is in the user's account. The trade or order that closes the user's position is referred to as a liquidation. 

- Some exchanges report **liquidations orders** in which they will report the creation of a liquidation order when a trader’s position initially enters liquidation. When a trader’s position enters liquidation, an exchange will typically enter a limit order at the trader's bankruptcy price. The order will show the amount of the position that is being liquidated and the liquidation price, but will not represent the matched trades that are executed as a result of the liquidation. 
- Other exchanges will report **liquidation trades** which represent the actual matched trades as a result of a liquidation order but will not report liquidation orders. 
- Some exchanges will report both liquidation orders and liquidation trades.

In [32]:
mkt_liqs_binance = client.get_market_liquidations(
    markets='binance-BTCUSDT-future', 
    limit_per_market=3
).to_dataframe()

In [33]:
mkt_liqs_binance.head()

,market,time,coin_metrics_id,amount,price,type,database_time,side
0,binance-BTCUSDT-future,2019-09-10 19:36:50.009000+00:00,1568144210009000000,0.199,10013.89,trade,2020-10-08 06:03:35.854962+00:00,sell
1,binance-BTCUSDT-future,2019-09-10 19:38:06.010000+00:00,1568144286010000000,0.04,9952.16,trade,2020-10-08 06:03:35.854962+00:00,sell
2,binance-BTCUSDT-future,2019-09-11 06:51:13.010000+00:00,1568184673010000000,0.04,9944.65,trade,2020-10-08 06:03:35.854962+00:00,sell


### Futures Liquidations - Aggregated Daily or Hourly


In [34]:
liqs_binance = client.get_market_metrics(
    markets='binance-BTCUSDT-future', 
    metrics='liquidations_reported_future_buy_usd_1d'
).to_dataframe()

In [35]:
liqs_binance.head()

,market,time,liquidations_reported_future_buy_usd_1d
0,binance-BTCUSDT-future,2020-01-01 00:00:00+00:00,457280.6609
1,binance-BTCUSDT-future,2020-01-02 00:00:00+00:00,168543.74943
2,binance-BTCUSDT-future,2020-01-03 00:00:00+00:00,10278909.782924
3,binance-BTCUSDT-future,2020-01-04 00:00:00+00:00,928114.69832
4,binance-BTCUSDT-future,2020-01-05 00:00:00+00:00,2481231.83264


**Fig. 7 - Bitcoin perpetual futures liquidations from State of the Market**    
<img src="https://5264302.fs1.hubspotusercontent-na1.net/hubfs/5264302/State%20of%20the%20Market%20-%20Chart%20Examples/BTC-Perp-Liquidations.png" width=450 margin-left='auto' margin-right='auto'/>

---
# Example 5: Options data types
---

We offer options data from two of the most liquid options trading venues, Deribit and OKX. Supported data types include implied volatility, trades, open interest, contract prices, contract specifications, quotes, and greeks. We recently expanded our options coverage to include several new data types from Deribit and added several new API endpoints to serve this data.

### Options Contracts - Implied Volatility

In [36]:
iv_deribit = client.get_market_implied_volatility(
    markets='deribit-BTC-18AUG22-*-option', 
    end_time='2022-08-17',
    limit_per_market=1
).to_dataframe()

In [37]:
iv_deribit.head()

,market,time,database_time,iv_bid,iv_ask,iv_mark,exchange_time
0,deribit-BTC-18AUG22-19000-C-option,2022-08-16 08:03:00+00:00,2022-08-16 08:03:26.923159+00:00,0.0,0.0,1.0,2022-08-16 08:03:25.946000+00:00
1,deribit-BTC-18AUG22-19000-P-option,2022-08-16 08:03:00+00:00,2022-08-16 08:03:26.923159+00:00,0.0,2.5531,1.0,2022-08-16 08:03:25.946000+00:00
2,deribit-BTC-18AUG22-20000-C-option,2022-08-16 08:03:00+00:00,2022-08-16 08:03:26.923159+00:00,0.0,0.0,1.0,2022-08-16 08:03:25.945000+00:00
3,deribit-BTC-18AUG22-20000-P-option,2022-08-16 08:03:00+00:00,2022-08-16 08:03:26.923159+00:00,0.0,2.0508,1.0,2022-08-16 08:03:24.938000+00:00
4,deribit-BTC-18AUG22-21000-C-option,2022-08-16 08:03:00+00:00,2022-08-16 08:03:26.923159+00:00,0.0,0.0,1.0,2022-08-16 08:03:25.950000+00:00


**Fig. 8 - Bitcoin 'Volatility Smile' from State of the Market**    
<img src="https://5264302.fs1.hubspotusercontent-na1.net/hubfs/5264302/State%20of%20the%20Market%20-%20Chart%20Examples/BTC-IV-vs-StrikePrice.png" width=800 margin-left='auto' margin-right='auto'/>

### Options Contracts - Market Greeks

In [38]:
greeks_deribit = client.get_market_greeks(
    markets='deribit-BTC-18AUG22-*-option', 
    end_time='2022-08-17',
    limit_per_market=1
).to_dataframe()

In [39]:
greeks_deribit.head()

,market,time,database_time,vega,theta,rho,delta,gamma,exchange_time
0,deribit-BTC-18AUG22-19000-C-option,2022-08-16 08:03:00+00:00,2022-08-16 08:03:26.923159+00:00,0.03933,-0.98434,1.03916,0.99937,0.0,2022-08-16 08:03:25.946000+00:00
1,deribit-BTC-18AUG22-19000-P-option,2022-08-16 08:03:00+00:00,2022-08-16 08:03:26.923159+00:00,0.03933,-0.30775,-0.00085,-0.00063,0.0,2022-08-16 08:03:25.946000+00:00
2,deribit-BTC-18AUG22-20000-C-option,2022-08-16 08:03:00+00:00,2022-08-16 08:03:26.923159+00:00,0.28905,-7.2338,1.08706,0.9943,0.00001,2022-08-16 08:03:25.945000+00:00
3,deribit-BTC-18AUG22-20000-P-option,2022-08-16 08:03:00+00:00,2022-08-16 08:03:26.923159+00:00,0.28905,-3.31867,-0.00768,-0.0057,0.00001,2022-08-16 08:03:24.938000+00:00
4,deribit-BTC-18AUG22-21000-C-option,2022-08-16 08:03:00+00:00,2022-08-16 08:03:26.923159+00:00,1.23369,-30.87443,1.10789,0.96931,0.00004,2022-08-16 08:03:25.950000+00:00


**Fig. 9 - Option Chain from State of the Market**    
<img src="https://5264302.fs1.hubspotusercontent-na1.net/hubfs/5264302/State%20of%20the%20Market%20-%20Chart%20Examples/Option-Chain.png" width=800 margin-left='auto' margin-right='auto'/>

### Options Contracts - Market Quotes

In [44]:
quotes_deribit = client.get_market_quotes(
    markets='deribit-BTC-18AUG22-*-option', 
    end_time='2022-08-17',
    limit_per_market=3
).to_dataframe()

In [45]:
quotes_deribit.head()

,market,time,coin_metrics_id,ask_price,ask_size,bid_price,bid_size
0,deribit-BTC-18AUG22-19000-C-option,2022-08-16 08:03:00+00:00,1660636980000000-0,0.0,0.0,0.0,0.0
1,deribit-BTC-18AUG22-19000-C-option,2022-08-16 08:04:00+00:00,1660637040000000-0,0.0,0.0,0.0,0.0
2,deribit-BTC-18AUG22-19000-C-option,2022-08-16 08:05:00+00:00,1660637100000000-0,0.0,0.0,0.0,0.0
3,deribit-BTC-18AUG22-19000-P-option,2022-08-16 08:03:00+00:00,1660636980000000-0,0.0085,0.1,0.0,0.0
4,deribit-BTC-18AUG22-19000-P-option,2022-08-16 08:04:00+00:00,1660637040000000-0,0.0085,0.1,0.0,0.0


### Options Contracts - Market Open Interest

In [42]:
oi_deribit = client.get_market_open_interest(
    markets='deribit-BTC-30DEC22-*-option', 
    paging_from='end',
    limit_per_market=1
).to_dataframe()

In [43]:
oi_deribit.sort_values('value_usd').tail()

,market,time,contract_count,value_usd,database_time,exchange_time
70,deribit-BTC-30DEC22-70000-C-option,2022-09-08 13:24:00+00:00,2793.2,53337606.464,2022-09-08 13:24:06.564572+00:00,2022-09-08 13:24:00+00:00
64,deribit-BTC-30DEC22-50000-C-option,2022-09-08 13:24:00+00:00,3237.2,61819157.428,2022-09-08 13:24:09.204524+00:00,2022-09-08 13:24:00+00:00
72,deribit-BTC-30DEC22-80000-C-option,2022-09-08 13:24:00+00:00,3570.7,68188122.499,2022-09-08 13:24:04.563095+00:00,2022-09-08 13:24:00+00:00
58,deribit-BTC-30DEC22-40000-C-option,2022-09-08 13:24:00+00:00,3953.2,75492244.268,2022-09-08 13:24:09.565375+00:00,2022-09-08 13:24:00+00:00
60,deribit-BTC-30DEC22-45000-C-option,2022-09-08 13:24:00+00:00,5133.6,98034357.096,2022-09-08 13:24:16.213028+00:00,2022-09-08 13:24:00+00:00


**Fig. 10 - Option Open Interest from State of the Market**    
<img src="https://5264302.fs1.hubspotusercontent-na1.net/hubfs/5264302/State%20of%20the%20Market%20-%20Chart%20Examples/Options-OI.png" width=800 margin-left='auto' margin-right='auto'/>